In [8]:
from fastembed import TextEmbedding
import numpy as np

In [2]:
model_handle = "jinaai/jina-embeddings-v2-small-en"


In [3]:
model = TextEmbedding(model_handle)


In [6]:
query = "I just discovered the course. Can I join now?"
embeddings_gen = model.embed([query])
embedding = next(embeddings_gen)


In [9]:
print("Embedding shape:", embedding.shape)
print("Minimum value in embedding:", np.min(embedding))


Embedding shape: (512,)
Minimum value in embedding: -0.11726373551188797


In [14]:
norm_q = np.linalg.norm(embedding)
print(norm_q)
dot_q_q = embedding.dot(embedding)
print("q · q =", dot_q_q)

1.0
q · q = 1.0


In [15]:
doc = "Can I still join the course after the start date?"
emb_gen = model.embed([query, doc])


In [16]:
# query vector
q_vec = next(emb_gen)
# document vector
d_vec = next(emb_gen)


In [18]:
cos_sim = q_vec.dot(d_vec)
print("Cosine similarity between query and document:", cos_sim)

Cosine similarity between query and document: 0.9008528856818037


In [23]:
query = "I just discovered the course. Can I join now?"
q = next(model.embed([query]))

In [19]:
documents = [
    {
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        "section": "General course-related questions",
        "question": "Course - Can I still join the course after the start date?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.",
        "section": "General course-related questions",
        "question": "Course - Can I follow the course after it finishes?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.",
        "section": "General course-related questions",
        "question": "Course - What can I do before the course starts?",
        "course": "data-engineering-zoomcamp",
    },
    {
        "text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.",
        "section": "General course-related questions",
        "question": "How can we contribute to the course?",
        "course": "data-engineering-zoomcamp",
    },
]

In [24]:
texts = [doc["text"] for doc in documents]
emb_gen = model.embed(texts)


In [25]:
V = np.stack([next(emb_gen) for _ in texts])


In [27]:
cosines = V.dot(q)
print("Cosine similarities for Q3:", cosines)


Cosine similarities for Q3: [0.76296845 0.81823784 0.80853973 0.71330785 0.73044995]


In [28]:
best_idx = np.argmax(cosines)
print("Best document index:", best_idx)


Best document index: 1


In [34]:
full_texts = [doc["question"] + " " + doc["text"] for doc in documents]
emb_full_gen = model.embed(full_texts)
V_full = np.stack([next(emb_full_gen) for _ in full_texts])
cos_full = V_full.dot(q)
print("Cosine similarities", cos_full)


Cosine similarities [0.8514543  0.84365941 0.84082872 0.77551577 0.80860079]


In [36]:
best_idx_full = np.argmax(cos_full)
if best_idx_full != best_idx:
    print(f"Different - Q3 best={best_idx}, Q4 best={best_idx_full}")
else:
    print("Same best document for Q3 and Q4.")


Different - Q3 best=1, Q4 best=0


In [37]:
model = TextEmbedding("BAAI/bge-small-en")


Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.63it/s]


In [39]:
dummy = next(model.embed(["Hello world"]))
print("Embedding shape:", dummy.shape)


Embedding shape: (384,)


In [40]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import requests

collection_name = "ml-zoomcamp-faq"
client = QdrantClient("http://localhost:6333")

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=384,
            distance=Distance.COSINE,
        ),
    )

In [41]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
documents_raw = requests.get(docs_url).json()


In [42]:
documents = []
for course in documents_raw:
    if course["course"] != "machine-learning-zoomcamp":
        continue
    for doc in course["documents"]:
        doc["course"] = course["course"]
        documents.append(doc)


In [43]:
model = TextEmbedding("BAAI/bge-small-en")
texts = [d["question"] + " " + d["text"] for d in documents]
emb_gen = model.embed(texts)

points = []
for idx, vec in enumerate(emb_gen):
    points.append(PointStruct(id=idx, vector=vec, payload={"text": texts[idx]}))

client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [44]:
print(f"Upserted {len(points)} documents into '{collection_name}'")

Upserted 375 documents into 'ml-zoomcamp-faq'


In [45]:
model = TextEmbedding("BAAI/bge-small-en")
query = "I just discovered the course. Can I join now?"
q_vec = next(model.embed([query]))

In [50]:
from qdrant_client import models
response = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text="I just discovered the course. Can I join now?",
        model="BAAI/bge-small-en"
    ),
    limit=1,
    with_payload=True,
)
top = response.points[0]
print("Top result score:", top.score)

Top result score: 0.8703172
